In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import time
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [2]:
plt.style.use('seaborn-bright')

## Import and Transform the Data

In [3]:
split = '2017-05-01'
def read_process_data(drop_cols):
    df = pd.read_csv('lag.csv',parse_dates=['date'])
    df.sort_values(by=['date','store_nbr','family'],inplace=True)
    df.reset_index(drop = True,inplace = True)
    df.drop(drop_cols,axis=1,inplace=True)
    df = create_dummies(df,['store_nbr','family'])
    month = df[df['date'] >= split].to_numpy()
    X_train,y_train,X_test,y_test = data_split(df)
    del df
    gc.collect()
    return X_train,y_train,X_test,y_test

In [4]:
def create_dummies(data,col_list):
    
    for col_name in col_list:
        
        drop_first = False
        if len(data[col_name].value_counts().index) == 2:
            drop_first = True
            
        tempdf = pd.get_dummies(data[col_name],drop_first = drop_first)
        
        if not drop_first:
            tempdf.columns = [col_name +'_' + str(col) for col in tempdf.columns]
        else:
            tempdf.columns = [col_name]
            
        data = data.drop(col_name,axis = 1).join(tempdf)
        
    return data

In [5]:
def data_split(data):
    # Split data into training and testing sets
    data_train = data[data['date'] < split]
    data_test  = data[data['date'] >= split]

    min_max_scaler = MinMaxScaler()
    
    mm = min_max_scaler.fit(data_train.drop('date',axis = 1))
    data_train = pd.DataFrame(mm.transform(data_train.drop('date',axis = 1)),columns=data_train.drop('date',axis = 1).columns, index=data_train.drop('date',axis = 1).index)
    data_test  = pd.DataFrame(mm.transform(data_test.drop('date',axis = 1)),columns=data_test.drop('date',axis = 1).columns, index=data_test.drop('date',axis = 1).index)
    
    # Split data into dependent and independent variables
    X_train = data_train.drop(['unit_sales'],axis = 1)
    y_train = data_train['unit_sales']
    X_test  = data_test.drop(['unit_sales'],axis = 1)
    y_test  = data_test['unit_sales']
    
    return X_train,y_train,X_test,y_test

In [6]:
X_train,y_train,X_test,y_test = read_process_data('Oil Price')

In [7]:
X_train.head()

,onpromotion,unit_sales (t-1),unit_sales (t-2),unit_sales (t-3),unit_sales (t-4),unit_sales (t-5),unit_sales (t-6),unit_sales (t-7),unit_sales (t-8),unit_sales (t-9),...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,0.0,0.000175,0.000292,0.001575,0.000175,0.000467,0.000817,0.000175,0.000991,0.001633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000233,0.000175,0.000292,0.001575,0.000175,0.000467,0.000817,0.000175,0.000991,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000991,0.000233,0.000175,0.000292,0.001575,0.000175,0.000467,0.000817,0.000175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000058,0.000991,0.000233,0.000175,0.000292,0.001575,0.000175,0.000467,0.000817,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000292,0.000058,0.000991,0.000233,0.000175,0.000292,0.001575,0.000175,0.000467,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
X_train.shape

(12463762, 112)

In [9]:
X_test.shape

(11320607, 112)

## Data Modeling

### Linear Support Vector Machine

In [10]:
lin_clf = SVR(kernel = 'linear',verbose=2)
begin = time.time()
lin_clf.fit(X_train, y_train)
end = time.time()

[LibSVM]

In [11]:
pred = lin_clf.predict(X_test)

In [12]:
print("The Mean Squared Error for the default Linear Support Vector Machine is: ",mean_squared_error(y_test,pred))
print(f"The computational effort for the default Linear Support Vector Machine in milliseconds is: {end - begin}")

The Mean Squared Error for the default Linear Support Vector Machine is:  0.00987362143707294
The computational effort for the default Linear Support Vector Machine in milliseconds is: 554.0276982784271


In [13]:
del pred
gc.collect()

20

### Optimization

In [23]:
param_grid = {'C': [0.001,0.007,0.01,0.015,0.1,0.3,0.5,0.7,0.9,1],
              'tol': [1e-3,1e-2,1e-4]
             }

gs = GridSearchCV(SVR(), param_grid, scoring='neg_mean_squared_error',cv = 4,verbose = 2)
gs.fit(X_train, y_train)

Fitting 4 folds for each of 30 candidates, totalling 120 fits
[CV] C=0.001, tol=0.001 ..............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............................... C=0.001, tol=0.001, total= 2.8min
[CV] C=0.001, tol=0.001 ..............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.8min remaining:    0.0s


[CV] ............................... C=0.001, tol=0.001, total= 2.7min
[CV] C=0.001, tol=0.001 ..............................................
[CV] ............................... C=0.001, tol=0.001, total= 1.8min
[CV] C=0.001, tol=0.001 ..............................................
[CV] ............................... C=0.001, tol=0.001, total= 1.8min
[CV] C=0.001, tol=0.01 ...............................................
[CV] ................................ C=0.001, tol=0.01, total= 2.2min
[CV] C=0.001, tol=0.01 ...............................................
[CV] ................................ C=0.001, tol=0.01, total= 2.2min
[CV] C=0.001, tol=0.01 ...............................................
[CV] ................................ C=0.001, tol=0.01, total= 1.8min
[CV] C=0.001, tol=0.01 ...............................................
[CV] ................................ C=0.001, tol=0.01, total= 1.7min
[CV] C=0.001, tol=0.0001 .............................................
[CV] .

[CV] ................................ C=0.1, tol=0.0001, total= 4.2min
[CV] C=0.3, tol=0.001 ................................................
[CV] ................................. C=0.3, tol=0.001, total= 3.4min
[CV] C=0.3, tol=0.001 ................................................
[CV] ................................. C=0.3, tol=0.001, total= 3.8min
[CV] C=0.3, tol=0.001 ................................................
[CV] ................................. C=0.3, tol=0.001, total= 2.6min
[CV] C=0.3, tol=0.001 ................................................
[CV] ................................. C=0.3, tol=0.001, total= 2.2min
[CV] C=0.3, tol=0.01 .................................................
[CV] .................................. C=0.3, tol=0.01, total= 2.0min
[CV] C=0.3, tol=0.01 .................................................
[CV] .................................. C=0.3, tol=0.01, total= 2.4min
[CV] C=0.3, tol=0.01 .................................................
[CV] .

[CV] .................................. C=1, tol=0.0001, total=25.2min
[CV] C=1, tol=0.0001 .................................................
[CV] .................................. C=1, tol=0.0001, total= 7.8min
[CV] C=1, tol=0.0001 .................................................
[CV] .................................. C=1, tol=0.0001, total=14.6min


[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 648.0min finished


GridSearchCV(cv=4, estimator=SVR(),
             param_grid={'C': [0.001, 0.007, 0.01, 0.015, 0.1, 0.3, 0.5, 0.7,
                               0.9, 1],
                         'tol': [0.001, 0.01, 0.0001]},
             scoring='neg_mean_squared_error', verbose=2)

In [24]:
for hps, values in gs.best_params_.items():
    print(f"{hps}: {values}")

C: 0.9
tol: 0.01


In [25]:
pred = gs.best_estimator_.predict(X_test)
print("The Mean Squared Error for the best Linear Support Vector Machine is: ",mean_squared_error(y_test,pred))

The Mean Squared Error for the best Linear Support Vector Machine is:  0.009861787525300446


### Non Linear Support Vector Machine

#### RBF

In [14]:
nlin_clf = SVR(kernel = 'rbf',verbose=2)
beginnl = time.time()
nlin_clf.fit(X_train, y_train)
endgnl = time.time()

[LibSVM]

In [15]:
pred = nlin_clf.predict(X_test)

In [16]:
print("The Mean Squared Error for the default RBF Support Vector Machine is: ",mean_squared_error(y_test,pred))
print(f"The computational effort for the default RBF Support Vector Machine in milliseconds is: {endgnl - beginnl}")

The Mean Squared Error for the default RBF Support Vector Machine is:  0.00976071727539594
The computational effort for the default RBF Support Vector Machine in milliseconds is: 328.1966609954834


### Optimization

In [26]:
param_grid = {'C': [0.001,0.007,0.01,0.015,0.1,0.3,0.5,0.7,0.9,1],
              'gamma': ['scale', 'auto'],
              'tol': [1e-2,1e-3,1e-4]
             }

gs = GridSearchCV(SVR(kernel = 'rbf'), param_grid, scoring='neg_mean_squared_error',cv = 4,verbose = 2)
gs.fit(X_train, y_train)

Fitting 4 folds for each of 60 candidates, totalling 240 fits
[CV] C=0.001, gamma=scale, tol=0.01 ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=0.001, gamma=scale, tol=0.01, total= 3.1min
[CV] C=0.001, gamma=scale, tol=0.01 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.1min remaining:    0.0s


[CV] ................... C=0.001, gamma=scale, tol=0.01, total= 2.5min
[CV] C=0.001, gamma=scale, tol=0.01 ..................................
[CV] ................... C=0.001, gamma=scale, tol=0.01, total= 1.7min
[CV] C=0.001, gamma=scale, tol=0.01 ..................................
[CV] ................... C=0.001, gamma=scale, tol=0.01, total= 1.7min
[CV] C=0.001, gamma=scale, tol=0.001 .................................
[CV] .................. C=0.001, gamma=scale, tol=0.001, total= 2.4min
[CV] C=0.001, gamma=scale, tol=0.001 .................................
[CV] .................. C=0.001, gamma=scale, tol=0.001, total= 2.6min
[CV] C=0.001, gamma=scale, tol=0.001 .................................
[CV] .................. C=0.001, gamma=scale, tol=0.001, total= 1.8min
[CV] C=0.001, gamma=scale, tol=0.001 .................................
[CV] .................. C=0.001, gamma=scale, tol=0.001, total= 1.9min
[CV] C=0.001, gamma=scale, tol=0.0001 ................................
[CV] .

...

[CV] .................... C=0.1, gamma=auto, tol=0.0001, total= 6.5min
[CV] C=0.1, gamma=auto, tol=0.0001 ...................................
[CV] .................... C=0.1, gamma=auto, tol=0.0001, total= 2.9min
[CV] C=0.1, gamma=auto, tol=0.0001 ...................................
[CV] .................... C=0.1, gamma=auto, tol=0.0001, total= 3.0min
[CV] C=0.3, gamma=scale, tol=0.01 ....................................
[CV] ..................... C=0.3, gamma=scale, tol=0.01, total= 2.0min
[CV] C=0.3, gamma=scale, tol=0.01 ....................................
[CV] ..................... C=0.3, gamma=scale, tol=0.01, total= 2.4min
[CV] C=0.3, gamma=scale, tol=0.01 ....................................
[CV] ..................... C=0.3, gamma=scale, tol=0.01, total= 1.8min
[CV] C=0.3, gamma=scale, tol=0.01 ....................................
[CV] ..................... C=0.3, gamma=scale, tol=0.01, total= 1.6min
[CV] C=0.3, gamma=scale, tol=0.001 ...................................
[CV] .

[CV] .................... C=0.7, gamma=scale, tol=0.001, total= 3.8min
[CV] C=0.7, gamma=scale, tol=0.0001 ..................................
[CV] ................... C=0.7, gamma=scale, tol=0.0001, total=17.9min
[CV] C=0.7, gamma=scale, tol=0.0001 ..................................
[CV] ................... C=0.7, gamma=scale, tol=0.0001, total=34.5min
[CV] C=0.7, gamma=scale, tol=0.0001 ..................................
[CV] ................... C=0.7, gamma=scale, tol=0.0001, total= 6.7min
[CV] C=0.7, gamma=scale, tol=0.0001 ..................................
[CV] ................... C=0.7, gamma=scale, tol=0.0001, total=14.6min
[CV] C=0.7, gamma=auto, tol=0.01 .....................................
[CV] ...................... C=0.7, gamma=auto, tol=0.01, total= 2.1min
[CV] C=0.7, gamma=auto, tol=0.01 .....................................
[CV] ...................... C=0.7, gamma=auto, tol=0.01, total= 2.3min
[CV] C=0.7, gamma=auto, tol=0.01 .....................................
[CV] .

[CV] ....................... C=1, gamma=auto, tol=0.001, total= 2.6min
[CV] C=1, gamma=auto, tol=0.001 ......................................
[CV] ....................... C=1, gamma=auto, tol=0.001, total= 1.6min
[CV] C=1, gamma=auto, tol=0.001 ......................................
[CV] ....................... C=1, gamma=auto, tol=0.001, total= 1.6min
[CV] C=1, gamma=auto, tol=0.0001 .....................................
[CV] ...................... C=1, gamma=auto, tol=0.0001, total=13.7min
[CV] C=1, gamma=auto, tol=0.0001 .....................................
[CV] ...................... C=1, gamma=auto, tol=0.0001, total=18.5min
[CV] C=1, gamma=auto, tol=0.0001 .....................................
[CV] ...................... C=1, gamma=auto, tol=0.0001, total= 8.0min
[CV] C=1, gamma=auto, tol=0.0001 .....................................
[CV] ...................... C=1, gamma=auto, tol=0.0001, total=10.2min


[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 1012.1min finished


GridSearchCV(cv=4, estimator=SVR(),
             param_grid={'C': [0.001, 0.007, 0.01, 0.015, 0.1, 0.3, 0.5, 0.7,
                               0.9, 1],
                         'gamma': ['scale', 'auto'],
                         'tol': [0.01, 0.001, 0.0001]},
             scoring='neg_mean_squared_error', verbose=2)

In [27]:
for hps, values in gs.best_params_.items():
    print(f"{hps}: {values}")

C: 0.9
gamma: scale
tol: 0.01


In [28]:
pred = gs.best_estimator_.predict(X_test)
print("The Mean Squared Error for the best RBF Support Vector Machine is: ",mean_squared_error(y_test,pred))

The Mean Squared Error for the best RBF Support Vector Machine is:  0.009861787525300446


#### Polynomial

In [17]:
nlin_clf = SVR(kernel = 'poly',verbose=2)
beginnl = time.time()
nlin_clf.fit(X_train, y_train)
endgnl = time.time()

[LibSVM]

In [18]:
pred = nlin_clf.predict(X_test)

In [19]:
print("The Mean Squared Error for the default Polynomial Support Vector Machine is: ",mean_squared_error(y_test,pred))
print(f"The computational effort for the default Polynomial Support Vector Machine in milliseconds is: {endgnl - beginnl}")

The Mean Squared Error for the default Polynomial Support Vector Machine is:  0.009907989409641996
The computational effort for the default Polynomial Support Vector Machine in milliseconds is: 535.6822106838226


### Optimization

In [10]:
param_grid = {'C': [0.001,0.007,0.01,0.015,0.1,0.3,0.5,0.7,0.9,1],
              'degree': [2,3,4,5],
              'gamma': ['scale', 'auto'],
             }

gs = GridSearchCV(SVR(kernel = 'poly'), param_grid, scoring='neg_mean_squared_error',cv = 4,verbose = 2)
gs.fit(X_train, y_train)

Fitting 4 folds for each of 80 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] C=0.001, degree=2, gamma=scale ..................................
[CV] ................... C=0.001, degree=2, gamma=scale, total= 3.9min
[CV] C=0.001, degree=2, gamma=scale ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.9min remaining:    0.0s


[CV] ................... C=0.001, degree=2, gamma=scale, total= 3.0min
[CV] C=0.001, degree=2, gamma=scale ..................................
[CV] ................... C=0.001, degree=2, gamma=scale, total= 2.4min
[CV] C=0.001, degree=2, gamma=scale ..................................
[CV] ................... C=0.001, degree=2, gamma=scale, total= 2.1min
[CV] C=0.001, degree=2, gamma=auto ...................................
[CV] .................... C=0.001, degree=2, gamma=auto, total= 2.4min
[CV] C=0.001, degree=2, gamma=auto ...................................
[CV] .................... C=0.001, degree=2, gamma=auto, total= 2.6min
[CV] C=0.001, degree=2, gamma=auto ...................................
[CV] .................... C=0.001, degree=2, gamma=auto, total= 1.7min
[CV] C=0.001, degree=2, gamma=auto ...................................
[CV] .................... C=0.001, degree=2, gamma=auto, total= 2.2min
[CV] C=0.001, degree=3, gamma=scale ..................................
[CV] .

...

[CV] ...................... C=0.3, degree=3, gamma=auto, total= 1.9min
[CV] C=0.3, degree=4, gamma=scale ....................................
[CV] ..................... C=0.3, degree=4, gamma=scale, total= 5.2min
[CV] C=0.3, degree=4, gamma=scale ....................................
[CV] ..................... C=0.3, degree=4, gamma=scale, total= 7.8min
[CV] C=0.3, degree=4, gamma=scale ....................................
[CV] ..................... C=0.3, degree=4, gamma=scale, total= 4.7min
[CV] C=0.3, degree=4, gamma=scale ....................................
[CV] ..................... C=0.3, degree=4, gamma=scale, total= 3.8min
[CV] C=0.3, degree=4, gamma=auto .....................................
[CV] ...................... C=0.3, degree=4, gamma=auto, total= 2.6min
[CV] C=0.3, degree=4, gamma=auto .....................................
[CV] ...................... C=0.3, degree=4, gamma=auto, total= 2.9min
[CV] C=0.3, degree=4, gamma=auto .....................................
[CV] .

[CV] ..................... C=0.7, degree=3, gamma=scale, total= 7.4min
[CV] C=0.7, degree=3, gamma=scale ....................................
[CV] ..................... C=0.7, degree=3, gamma=scale, total= 2.7min
[CV] C=0.7, degree=3, gamma=scale ....................................
[CV] ..................... C=0.7, degree=3, gamma=scale, total= 3.3min
[CV] C=0.7, degree=3, gamma=auto .....................................
[CV] ...................... C=0.7, degree=3, gamma=auto, total= 1.9min
[CV] C=0.7, degree=3, gamma=auto .....................................
[CV] ...................... C=0.7, degree=3, gamma=auto, total= 2.0min
[CV] C=0.7, degree=3, gamma=auto .....................................
[CV] ...................... C=0.7, degree=3, gamma=auto, total= 1.4min
[CV] C=0.7, degree=3, gamma=auto .....................................
[CV] ...................... C=0.7, degree=3, gamma=auto, total= 1.7min
[CV] C=0.7, degree=4, gamma=scale ....................................
[CV] .

[CV] ....................... C=1, degree=2, gamma=scale, total= 3.0min
[CV] C=1, degree=2, gamma=auto .......................................
[CV] ........................ C=1, degree=2, gamma=auto, total= 2.1min
[CV] C=1, degree=2, gamma=auto .......................................
[CV] ........................ C=1, degree=2, gamma=auto, total= 2.0min
[CV] C=1, degree=2, gamma=auto .......................................
[CV] ........................ C=1, degree=2, gamma=auto, total= 1.4min
[CV] C=1, degree=2, gamma=auto .......................................
[CV] ........................ C=1, degree=2, gamma=auto, total= 1.7min
[CV] C=1, degree=3, gamma=scale ......................................
[CV] ....................... C=1, degree=3, gamma=scale, total= 4.4min
[CV] C=1, degree=3, gamma=scale ......................................
[CV] ....................... C=1, degree=3, gamma=scale, total= 8.6min
[CV] C=1, degree=3, gamma=scale ......................................
[CV] .

[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed: 1006.4min finished


GridSearchCV(cv=4, estimator=SVR(kernel='poly'),
             param_grid={'C': [0.001, 0.007, 0.01, 0.015, 0.1, 0.3, 0.5, 0.7,
                               0.9, 1],
                         'degree': [2, 3, 4, 5], 'gamma': ['scale', 'auto']},
             scoring='neg_mean_squared_error', verbose=2)

In [11]:
for hps, values in gs.best_params_.items():
    print(f"{hps}: {values}")

C: 1
degree: 5
gamma: scale


In [12]:
pred = gs.best_estimator_.predict(X_test)
print("The Mean Squared Error for the best Polynomial Support Vector Machine is: ",mean_squared_error(y_test,pred))

The Mean Squared Error for the best Polynomial Support Vector Machine is:  0.00990053669671002
